In [ ]:
from matplotlib import pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_csv('drive/My Drive/churn-data-3333.csv')

In [ ]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)
cat_df=cat_df.drop(['phone number'], axis=1)


num_df=num_df.fillna(conData)

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn']
y=churn



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
train=pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
#train=pd.concat([X, y], axis=1)
#test=pd.concat([X, y], axis=1)
X_train=X
X_test=X
y_train=y
y_test=y

In [ ]:
from google.colab import files
import os, sys
file_path = 'drive/My Drive/Colab Notebooks/private-data-generation/'
#sys.path.append(file_path)
sys.path.append(os.path.abspath(file_path))
import sys
sys.path.insert(1,'/content/drive/My Drive/Colab Notebooks/private-data-generation/')

In [ ]:
from models import dp_wgan, pate_gan, ron_gauss

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn import preprocessing
from scipy.special import expit
from models import dp_wgan, pate_gan, ron_gauss
from models.Private_PGM import private_pgm
import argparse
import numpy as np
import pandas as pd
import collections
import os

In [ ]:
class  Class_opt:
    "This is a person class"
    categorical = False
    model='dp-wgan'
    target_epsilon=80
    target_delta=0.8
    downstream_task="classification"
    target_variable='status'
    batch_size=64
    micro_batch_size=8
    clamp_lower=0.01
    clamp_upper=0.01
    clip_coeff=0.1
    sigma=2.0
    num_epochs=50
    enable_privacy=0




opt = Class_opt()
opt.target_variable='churn'
opt.categorical = True
opt.model='dp-wgan'
opt.target_epsilon=10
opt.target_delta=0.1
opt.batch_size=64
opt.micro_batch_size=8
opt.clamp_lower=0.01
opt.clamp_upper=0.01
opt.clip_coeff=0.1
opt.sigma=0.8
opt.num_epochs=200
opt.enable_privacy=1

opt.downstream_task="classification"

In [ ]:
data_columns = [col for col in train.columns if col != opt.target_variable]

In [ ]:
class_ratios = train[opt.target_variable].sort_values().groupby(train[opt.target_variable]).size().values/train.shape[0]
class_ratios

array([0.85426489, 0.14573511])

In [ ]:
X_train = np.nan_to_num(train.drop([opt.target_variable], axis=1).values)
y_train = np.nan_to_num(train[opt.target_variable].values)
X_test = np.nan_to_num(test.drop([opt.target_variable], axis=1).values)
y_test = np.nan_to_num(test[opt.target_variable].values)

In [ ]:
#Normalized the data
X_train = expit(X_train)
X_test = expit(X_test)

In [ ]:
input_dim = X_train.shape[1]
z_dim = int(input_dim / 4 + 1) if input_dim % 4 == 0 else int(input_dim / 4)

Hyperparams = collections.namedtuple('Hyperarams','batch_size micro_batch_size clamp_lower clamp_upper clip_coeff sigma class_ratios lr num_epochs')
Hyperparams.__new__.__defaults__ = (None, None, None, None, None, None, None, None, None)

In [ ]:
conditional = (opt.downstream_task == "classification")
model = dp_wgan.DP_WGAN(input_dim, z_dim, opt.target_epsilon, opt.target_delta, conditional)
model.train(X_train, y_train, Hyperparams(batch_size=opt.batch_size, micro_batch_size=opt.micro_batch_size,
                                              clamp_lower=opt.clamp_lower, clamp_upper=opt.clamp_upper,
                                              clip_coeff=opt.clip_coeff, sigma=opt.sigma, class_ratios=class_ratios, lr=
                                              5e-5, num_epochs=opt.num_epochs), private=opt.enable_privacy)

torch.save(model, 'drive/My Drive/chkp_GAN_MWOE_privacy_1_70-vs-30_dataset3333.pth')

Epoch : 1 Loss D real :  0.026622602264132742 Loss D fake :  0.010916086580844538 Loss G :  0.011101581368358803 Epsilon spent :  4.219200930360318
Epoch : 2 Loss D real :  0.026553565666404103 Loss D fake :  0.010770680155900457 Loss G :  0.010952642903704422 Epsilon spent :  4.6007600390638945
Epoch : 3 Loss D real :  0.026644799748791668 Loss D fake :  0.010909441259536256 Loss G :  0.010999148067228586 Epsilon spent :  4.98231914776747
Epoch : 4 Loss D real :  0.02659486628575705 Loss D fake :  0.010966114517138605 Loss G :  0.011030759180744115 Epsilon spent :  5.363878256471046
Epoch : 5 Loss D real :  0.026827310827977863 Loss D fake :  0.01082840144463874 Loss G :  0.01092543188164361 Epsilon spent :  5.745437365174622
Epoch : 6 Loss D real :  0.02670233867086363 Loss D fake :  0.010856446588374587 Loss G :  0.01091548113554381 Epsilon spent :  6.126996473878197
Epoch : 7 Loss D real :  0.026499412077766897 Loss D fake :  0.010949788168767087 Loss G :  0.010889967429272174 Epsi

In [ ]:
if opt.model == 'imle' or opt.model == 'dp-wgan' or opt.model == 'pate-gan':
    syn_data = model.generate(X_train.shape[0], class_ratios)
    X_syn, y_syn = syn_data[:, :-1], syn_data[:, -1]

In [ ]:
X_syn_df = pd.DataFrame(data=X_syn, columns=data_columns)
y_syn_df = pd.DataFrame(data=y_syn, columns=[opt.target_variable])

syn_df = pd.concat([X_syn_df, y_syn_df], axis=1)
syn_df.to_csv("drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset3333.csv")
test.to_csv("drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset3333.csv")
print("Saved synthetic data at : ", "drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70-percent-traning_dataset3333.csv")

Saved synthetic data at :  drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70-percent-traning_dataset3333.csv


In [ ]:
# Creating downstream learners
learners = []
if opt.downstream_task == "classification":
    names = ['LR', 'Random Forest', 'Neural Network', 'GaussianNB', 'GradientBoostingClassifier', 'DT', 'KNN', 'SVN']

    learners.append((LogisticRegression()))
    learners.append((RandomForestClassifier()))
    learners.append((MLPClassifier(early_stopping=True)))
    learners.append((GaussianNB()))
    learners.append((GradientBoostingClassifier()))
    learners.append(DecisionTreeClassifier())
    learners.append((KNeighborsClassifier()))
    #learners.append((svm.SVC()))

    print("AUC scores of downstream classifiers on test data : ")
    for i in range(0, len(learners)):
        score = learners[i].fit(X_syn, y_syn)
        #score = learners[i].fit(X_train, y_train)
        pred_probs = learners[i].predict_proba(X_test)
        auc_score = roc_auc_score(y_test, pred_probs[:, 1])
        print('-' * 40)
        print('{0}: {1}'.format(names[i], auc_score))

AUC scores of downstream classifiers on test data : 
----------------------------------------
LR: 0.4927009979518731
----------------------------------------
Random Forest: 0.4883966675098531
----------------------------------------
Neural Network: 0.42329315958254116
----------------------------------------
GaussianNB: 0.4860343856843274
----------------------------------------
GradientBoostingClassifier: 0.5930755358993399
----------------------------------------
DT: 0.5
----------------------------------------
KNN: 0.5332677823926366


In [ ]:
import numpy as np
class WOE_Encoder():
    def __init__(self, cols=None, size=None):
            self.cols = cols
            self.min_samples=1
            #self.bins=10000
            self.bins=int(size/10)
            self._mapping = {}

    def WOE_fit(self, X, y):
        for col in self.cols:
            X[col]=X[col].fillna(-9999)
            if (len(np.unique(X[col]))>100):
                binned_x = pd.qcut(X[col], self.bins,  duplicates='drop')
                d0 = pd.DataFrame({'x': binned_x, 'y':y})
            else:
                d0 = pd.DataFrame({'x': X[col], 'y': y})
            #print (d0)
            # Share of positive (resp. negative) labels for each category P(X=X_i | Y=1) (resp. P(X=X_i | Y=0))
            #mapping = y.groupby(X[col]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping = y.groupby(d0["x"]).agg(['sum', 'count']).rename({'sum': 'pos'}, axis=1)
            mapping['neg'] = mapping['count'] - mapping['pos']
            mapping[['pos', 'neg']] /= mapping[['pos', 'neg']].sum()
            # For corner cases, defaulting to WOE = 0 (meaning no info). To avoid division by 0 we use default values.
            undef = (mapping['count'] < self.min_samples) | (mapping['pos'] == 0) | (mapping['neg'] == 0)
            mapping.loc[undef, ['pos', 'neg']] = -1
            # Final step, log of ratio of probabily estimates
            mapping['value'] = np.log((mapping['pos'] +0.0001)/ (mapping['neg']+0.0001))
            self._mapping[col] = mapping


        X_encoded = X.copy(deep=True)
        for col, mapping in self._mapping.items():
            X_encoded.loc[:, col] = X_encoded[col].fillna(-9999).map(mapping['value'])
            X_encoded[col].fillna(0, inplace=True)

        return X_encoded

In [ ]:
syn_df=pd.read_csv('drive/My Drive/synthetic_data_GAN_mWOE_privacy_1_70percent_training_dataset3333.csv')
y_syn_df=syn_df['churn']
X_syn_df=syn_df.drop(['churn'], axis=1)
Obj = WOE_Encoder(cols=X_syn_df.columns, size=X_syn_df.shape[0])
X_encoded = Obj.WOE_fit(X_syn_df, y_syn_df)
X_encoded.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,-0.444907,0.450928,0.450928,0.000000,1.813154,0.981190,0.000000,0.450928,0.000000,0.000000,0.000000,0.981190,0.450928,0.000000,-0.341739,-0.341739,-0.341739,0.981190,0.000000,0.000000
1,-0.444907,-0.341739,0.000000,-0.341739,0.000000,-0.341739,0.000000,-0.341739,0.000000,0.000000,0.000000,-0.341739,0.450928,0.450928,0.000000,-0.341739,0.000000,0.000000,0.000000,0.000000
2,-0.444907,-0.341739,0.450928,1.415745,-0.341739,0.000000,0.450928,0.981190,-0.341739,0.450928,-0.341739,0.000000,-0.341739,0.000000,-0.341739,0.000000,1.813154,0.000000,-0.341739,1.813154
3,-0.444907,0.000000,0.000000,0.000000,0.450928,0.450928,-0.341739,-0.341739,0.000000,0.450928,0.000000,1.415745,0.000000,0.981190,0.981190,0.981190,0.000000,-0.341739,2.207927,0.000000
4,-0.444907,-0.341739,0.000000,0.000000,0.000000,-0.341739,0.450928,0.000000,0.000000,0.000000,0.000000,0.000000,0.450928,0.000000,1.813154,-0.341739,0.981190,0.450928,0.000000,1.415745


In [ ]:
org_test_df=pd.read_csv('drive/My Drive/Original_data_GAN_mWOE_privacy_1_30percent_testing_dataset3333.csv')
y_test=org_test_df['churn']
X_test=org_test_df.drop(['churn'], axis=1)
Obj_test = WOE_Encoder(cols=X_test.columns, size=X_test.shape[0])
X_encoded_test = Obj_test.WOE_fit(X_test, y_test)
X_encoded_test.head()

,Unnamed: 0,account_length,area_code,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,international_plan,voice_mail_plan
0,0.000000,0.085209,-0.186013,0.201163,-0.285305,0.000000,-0.285305,0.400768,0.535296,0.400768,-0.600864,0.488305,-0.600864,0.178359,0.088739,0.178359,-0.397455,-0.458074,-0.221764,0.201163
1,1.374515,-0.401910,0.029383,0.201163,-0.285305,0.180214,-0.285305,0.000000,0.397299,0.000000,-0.600864,0.678013,-0.600864,-0.285305,-0.033924,-0.285305,-0.322192,-1.033450,-0.221764,0.201163
2,1.776454,-0.287066,0.029383,0.201163,-0.401910,1.222378,-0.401910,-0.285305,-0.401910,-0.285305,0.400768,0.248801,0.400768,-0.401910,0.454768,-0.401910,1.820992,0.582842,-0.221764,0.201163
3,-0.401910,-0.687231,0.121389,0.201163,0.000000,0.000000,0.000000,0.179746,-0.458074,0.179746,0.000000,-0.404257,0.000000,0.000000,0.466248,0.000000,-0.397455,-0.840368,-0.221764,0.201163
4,0.400768,-0.506327,0.121389,0.201163,0.000000,-0.200969,0.000000,0.532786,0.869338,0.532786,0.000000,-0.542787,0.000000,-0.648348,-0.772818,-0.648348,-0.397455,-0.458074,-0.221764,0.201163


In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=6.579332246575682e-09)

classifier.fit(X_encoded,y_syn_df)


cv_method = RepeatedKFold(n_splits=10,
                          n_repeats=3,
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 5)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


841 16 58 85
pod:  0.5944055944055944
pof:  0.01866977829638273
AUC:  0.7878679080546058
recall:  0.5944055944055944
precision:  0.8415841584158416
F1-Score:  0.6967213114754097
accuracy:  0.926


In [ ]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X_encoded, y = y_syn, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)



854 3 72 71
pod:  0.4965034965034965
pof:  0.003500583430571762
AUC:  0.7465014565364624
recall:  0.4965034965034965
precision:  0.9594594594594594
F1-Score:  0.6543778801843319
accuracy:  0.925


In [ ]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap= True, max_depth= 80, max_features= 3, min_samples_leaf= 3, min_samples_split= 12, n_estimators= 1000)
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


857 0 127 16
pod:  0.11188811188811189
pof:  0.0
AUC:  0.5559440559440559
recall:  0.11188811188811189
precision:  1.0
F1-Score:  0.20125786163522014
accuracy:  0.873


In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(metric='manhattan', weights='uniform', n_neighbors=19 )
classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


857 0 138 5
pod:  0.03496503496503497
pof:  0.0
AUC:  0.5174825174825175
recall:  0.03496503496503497
precision:  1.0
F1-Score:  0.06756756756756757
accuracy:  0.862


In [ ]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

#classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier = DecisionTreeClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


838 19 82 61
pod:  0.42657342657342656
pof:  0.022170361726954493
AUC:  0.7022015324232361
recall:  0.42657342657342656
precision:  0.7625
F1-Score:  0.547085201793722
accuracy:  0.899


In [ ]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(X_encoded,y_syn_df)
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


856 1 93 50
pod:  0.34965034965034963
pof:  0.0011668611435239206
AUC:  0.6742417442534129
recall:  0.34965034965034963
precision:  0.9803921568627451
F1-Score:  0.5154639175257731
accuracy:  0.906


In [ ]:
#XGBClassifier
from xgboost import XGBClassifier
classifier =  XGBClassifier(random_state=0)

classifier.fit(X_encoded,y_syn_df)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
#y_pred  = cross_val_predict(estimator = classifier, X = X, y = y, cv = 10)
y_pred = classifier.predict(X_encoded_test)
tn, fp, fn, tp  = confusion_matrix(y_test, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


856 1 91 52
pod:  0.36363636363636365
pof:  0.0011668611435239206
AUC:  0.68123475124642
recall:  0.36363636363636365
precision:  0.9811320754716981
F1-Score:  0.5306122448979592
accuracy:  0.908


In [ ]:
#FNN
from keras.models import Sequential
from keras.layers import Dense
#create model
model = Sequential()
#get number of columns in training data
n_cols = X_encoded_test.shape[1]
model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
#compile model using mse as a measure of model performance
#model.compile(optimizer='adam', loss='mean_squared_error')
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
from keras.callbacks import EarlyStopping
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=3)
#train model

model.fit(X_encoded, y_syn_df, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
y_pred = model.predict(X_encoded_test)
y22_pred=y_pred.round()
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp  = confusion_matrix(y_test, y22_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)


recall = tp/(tp+fn)
print ('recall: ',recall)
precision=tp/(tp+fp)
print ('precision: ',precision)

F1_score=(2*precision*recall)/(precision+recall)
print ('F1-Score: ',F1_score)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

Epoch 1/30
59/59 [==============================] - 3s 9ms/step - loss: 0.3326 - accuracy: 0.8773 - val_loss: 0.2071 - val_accuracy: 0.9229
Epoch 2/30
59/59 [==============================] - 0s 6ms/step - loss: 0.1725 - accuracy: 0.9346 - val_loss: 0.1535 - val_accuracy: 0.9593
Epoch 3/30
59/59 [==============================] - 0s 5ms/step - loss: 0.1388 - accuracy: 0.9512 - val_loss: 0.1318 - val_accuracy: 0.9572
Epoch 4/30
59/59 [==============================] - 0s 6ms/step - loss: 0.1215 - accuracy: 0.9561 - val_loss: 0.1187 - val_accuracy: 0.9615
Epoch 5/30
59/59 [==============================] - 0s 6ms/step - loss: 0.1039 - accuracy: 0.9582 - val_loss: 0.1069 - val_accuracy: 0.9593
Epoch 6/30
59/59 [==============================] - 0s 6ms/step - loss: 0.0948 - accuracy: 0.9657 - val_loss: 0.1255 - val_accuracy: 0.9593
Epoch 7/30
59/59 [==============================] - 0s 6ms/step - loss: 0.0911 - accuracy: 0.9678 - val_loss: 0.1000 - val_accuracy: 0.9700
Epoch 8/30
59/59 [==